# Cryptopals

`cryptopals.com` provides a complete series of exercices used to learn **cryptography**.

Of course we do not have the right to use any crypto tool or library of any kind !

# Set 1 : Basics

## Utils

All utilities used for Set 1.

In [23]:
import requests

In [12]:
def hex_to_bytes(input: str) -> list:
    """
    returns @input as bytes integers array
    
    @input must be an hex string
    """
    bytes = []
    for i in range(0, len(input), 2):
        bytes.append(int(input[i:i+2], 16))
    return bytes


def get_array_mean(array: list) -> int:
    """
    returns the mean of the integers array @array
    """
    if not len(array):
        return -1
    return sum(array) / len(array)


def str_to_hexstr(string: str) -> hex:
    """
    returns hexstring representation of @string
    """
    return "".join("{:02x}".format(ord(c)) for c in string)

## Hex to Base64

Always operate on raw bytes, never on encoded strings. Only use hex and base64 for pretty-printing.

In [25]:
BASE64_ALPHABET = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/"
SET1STEP1_INPUT = "49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d"
SET1STEP1_OUTPUT = "SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t"

In [26]:
def hex_to_b64(input: str) -> str:
    """
    returns the base64 encoded @input
    """
    result = ""
    bytes = hex_to_bytes(input)
    for i in range(0, len(bytes), 3):
        concat = bytes[i] << 16 | bytes[i + 1] << 8 | bytes[i + 2]
        chars = [concat >> 18, concat >> 12 & 0x3f, concat >> 6 & 0x3f, concat & 0x3f]
        for char in chars:
            result += BASE64_ALPHABET[char]
    return result

In [27]:
assert hex_to_b64(SET1STEP1_INPUT) == SET1STEP1_OUTPUT

## Fixed xor

Write a function that takes two equal-length buffers and produces their XOR combination.

In [28]:
SET1STEP2_INPUT = "1c0111001f010100061a024b53535009181c"
SET1STEP2_KEY = "686974207468652062756c6c277320657965"
SET1STEP2_OUTPUT = "746865206b696420646f6e277420706c6179"

In [29]:
def xor(input: str, key: str) -> str:
    """
    xor @input with @key
    
    @input @key and return value are hex strings
    """
    return hex(int(input, 16) ^ int(key, 16)).replace("0x", "")

In [30]:
assert xor(SET1STEP2_INPUT, SET1STEP2_KEY) == SET1STEP2_OUTPUT

## Single-byte xor cipher

Input as been XOR'd against a single character. Find the key, decrypt the message.

In [31]:
SET1STEP3_INPUT = "1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736"

In [32]:
def check_english(input: str) -> bool:
    """
    checks with frequency analysis if @input is an english sentence
    
    returns True if english language is detected
    @input must be a string
    """
    error_size = 2
    valid = 0
    english = [
        {"letter": "e", "frequency": 12.702},
        {"letter": "t", "frequency": 9.056},
        {"letter": "a", "frequency": 8.167},
        {"letter": "o", "frequency": 7.507},
        {"letter": "i", "frequency": 6.966},
        {"letter": "n", "frequency": 6.749}
    ]
    string = input.lower()
    for element in english:
        number = string.count(element.get("letter")) * 100 / len(string)
        frequency = element.get("frequency")
        if number >= (frequency - 0.3 * frequency) and number <= frequency + 0.3 * frequency:
            valid += 1
    if "'s " in string:
        valid += 2
    if valid >= len(english) - error_size:
        return True
    return False


def uncipher_single_byte_xor(input: str) -> str or None:
    """
    finds the single-byte key used to encrypt @input
    
    @input and return values are hex strings
    """
    bytes = hex_to_bytes(input)
    res = ""
    stock = ""
    for key in range(0, 255):
        for c in bytes:
            char = c ^ key
            if not (char > 30 and char < 128):
                break
            res += chr(char)
        if len(res) == len(bytes) and check_english(res):
            print("[+] Key is", key)
            stock = res
        res = ''
    return stock

In [33]:
print("[+] Decoded:", uncipher_single_byte_xor(SET1STEP3_INPUT))

[+] Key is 88
[+] Decoded: Cooking MC's like a pound of bacon


## Detect single-character XOR

Find the encrypted string with single byte xor.

In [34]:
SET1CHALL5_URL = "https://cryptopals.com/static/challenge-data/4.txt"

In [35]:
def detect_single_xor(tab: list) -> str:
    """
    returns the decoded string encoded with a single byte xor
    
    @tab must be a list of hex strings
    """
    for value in tab:
        res = uncipher_single_byte_xor(value)
        if res:
            print("[+] Decoded:", res)

In [36]:
r = requests.get(SET1CHALL5_URL)
tab = r.text.split("\n")
detect_single_xor(tab)

[+] Key is 99
[+] Decoded: T2XNi-]1rTIYbP/>2`o%%]T5JonmqA
[+] Key is 120
[+] Decoded: mjOia}tti:\"x7{N(0`H[ra]p$bo_^


## Implement repeating-key XOR

In repeating-key XOR, you'll sequentially apply each byte of the key.

In [38]:
SET1STEP6_INPUT = "Burning 'em, if you ain't quick and nimble\nI go crazy when I hear a cymbal"

SET1STEP6_KEY = "ICE"

SET1STEP6_OUTPUT = "0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f"

In [38]:
def repeating_xor_encrypt(string: str, key: str) -> str:
    """
    encrypt @input with @key
    
    @input and @key must be basic strings
    """
    res = ""
    for i in range(len(string)):
        tmp = hex(ord(string[i]) ^ ord(key[i % len(key)])).replace("0x", "")#
        res += (2 - len(tmp)) * "0" + tmp
    return res 

In [39]:
assert repeating_xor_encrypt(SET1STEP6_INPUT, SET1STEP6_KEY) == SET1STEP6_OUTPUT

# Break repeating-key XOR

###  Hamming distance

In [18]:
def get_hamming_distance(s1: hex, s2: hex) -> int:
    """
    returns the hamming sistance between @s1 and @s2
    
    @s1 and @s2 must be basic strings and same lengths
    """
    s1 = hex_to_bytes(s1)
    s2 = hex_to_bytes(s2)
    distance = 0
    for i in range(len(s1)):
        xor = s1[i] ^ s2[i]
        distance += str(bin(xor)).count("1")
    return distance

In [19]:
assert get_hamming_distance(str_to_hexstr("this is a test"), str_to_hexstr("wokka wokka!!!")) == 37

### Guessing KEYSIZE

In [44]:
def get_nsmallest(array: list, n: int) -> list:
    """
    returns a list composed of the @nth smallest elements of @array
    """
    for i in range(len(array) - 1, -1, -1):
        if array[i] == -1:
            del array[i]
    res = []
    save = array
    values = [i + 1 for i in range(len(save))]
    while n > 0 and len(array):
        index = save.index(min(array))
        res.append(values[array.index(min(array))])
        del values[array.index(min(array))]
        del array[array.index(min(array))]
        n -= 1
    return res


def guess_keysize(string: str, nb: int) -> int:
    """
    returns the @nb smallest guessed key sizes of encoded @string (hex string)
    """
    distances = []
    for i in range(1, 41):
        values = [string[j:j+i] for j in range(0, len(string), i)]
        tmp = []
        while len(values):
            try:
                tmp.append(get_hamming_distance(values[0], values[1]) / i)
            except IndexError:
                break
            values = values[1:]
        distances.append(get_array_mean(tmp))
    return get_nsmallest(distances, nb)

### Cracking the code

In [35]:
def crack_with_keysize(string: str, key_size: int) -> str:
    """
    finds the key of size key_size and decrypts @string
    
    @string must be an hex string
    """
    values = [string[j:j+key_size] for j in range(0, len(string), key_size)]
    print(values)

In [43]:
sizes = guess_keysize(SET1STEP6_OUTPUT, 2)
crack_with_keysize(SET1STEP6_OUTPUT, sizes[0])

['0b3637272a2b2e6362', '2c2e69692a23693a2a', '3c6324202d623d6334', '3c2a26226324272765', '272a282b2f20430a65', '2e2c652a3124333a65', '3e2b2027630c692b20', '283165286326302e27', '282f']
